In [26]:
import pandas as pd
import re

# Exemplo de DataFrame
data = {
    "Coluna1": ["DI1F27 DI1G28", "DI1X26", "Texto aleatório", "DI1H29 DI1Z30 DI1M31"],
    "Coluna2": ["Texto", "DI1K30", "Mais DI1F27 DI1K30", "Outro texto"]
}
df = pd.DataFrame(data)

# Função para encontrar vencimentos DI1 e adicionar colunas
def process_di1_symbols(df):
    # Padrão de vencimentos DI1 (DI1 seguido de uma letra e dois dígitos ou apenas uma letra seguida de dois dígitos)
    pattern = r"(DI1[A-Z]\d{2}|[A-Z]\d{2})"

    # Novas colunas
    subtipo = []
    simbolos_dict = {f"Simbolo {i+1}": [] for i in range(4)}

    # Iterar sobre cada linha
    for _, row in df.iterrows():
        found = []

        # Checar cada célula da linha
        for cell in row:
            if isinstance(cell, str):
                matches = re.findall(pattern, cell)
                # Adicionar prefixo DI1 caso esteja ausente
                matches = [f"DI1{match}" if not match.startswith("DI1") else match for match in matches]
                found.extend(matches)

        # Remover duplicados mantendo a ordem
        found = list(dict.fromkeys(found))

        # Adicionar subtipo baseado na quantidade de aparições
        if len(found) == 2:
            subtipo.append("FRA")
        elif len(found) == 3:
            subtipo.append("FlyFRA")
        elif len(found) == 4:
            subtipo.append("SlideFRA")
        else:
            subtipo.append(f"Apareceu {len(found)} vez(es)")

        # Preencher colunas de símbolos
        for i in range(4):
            simbolos_dict[f"Simbolo {i+1}"].append(found[i] if i < len(found) else None)

    # Adicionar colunas ao DataFrame
    df["Subtipo"] = subtipo
    for col_name, values in simbolos_dict.items():
        df[col_name] = values

    # Manter apenas as colunas criadas
    columns_to_keep = ["Subtipo"] + list(simbolos_dict.keys())
    df = df[columns_to_keep]

    return df

# Processar o DataFrame
result_df = process_di1_symbols(df)
display(result_df)

,Subtipo,Simbolo 1,Simbolo 2,Simbolo 3,Simbolo 4
0,FRA,DI1F27,DI1G28,None,None
1,FRA,DI1X26,DI1K30,None,None
2,FRA,DI1F27,DI1K30,None,None
3,FlyFRA,DI1H29,DI1Z30,DI1M31,None
